# In the Name of God
Name : Mohammad Mohajel Sadegi

SID  : 810199483


## Abstract:
Implementation of BFS, IDS and A* search 

## Importing libraries:

In [6]:
import copy
from collections import Counter

## Class graph:

In [2]:
class Graph:
    def __init__(self, v :int) -> None:
        self.v = v
        self.edges = [[] for _ in range(v)]
        self.morids = None #make it dictionary
        self.recipes = []
        self.start_position = 0
        self.rocky_nodes = []


    def add_edge(self, u :int, v :int) -> None:
        self.edges[u].append(v)
        self.edges[v].append(u)

    def is_rocky(self, node :int) -> bool:
        return (node in self.rocky_nodes)

    def is_recipes(self, node :int) -> bool:
        return (node in self.recipes)

    def is_morid(self, node :int) -> bool:
        return (node in list(self.morids.keys()))


## Class state

In [3]:
class State:
    def __init__(self, rocky_nodes) -> None:
        self.position = 0
        self.recipes_seen = []
        self.morids_seen = []
        self.prev_state = None
        self.rocky_nodes = dict.fromkeys(rocky_nodes, 0)
        self.rocky_remain = 0
        self.cost = 0

## Read input and create graph class

In [4]:
def create_graph(path :str) -> Graph:
    file = open(path, "r")
    n, m = map(int, file.readline().split(" "))
    g = Graph(n)

    for _ in range(m):
        u, v = map(int, file.readline().split(" "))
        g.add_edge(u - 1, v - 1)

    h = int(file.readline())
    rocky_nodes = list(map(int, file.readline().split(" ")))
    g.rocky_nodes = rocky_nodes
    s = int(file.readline())

    morids = {}
    for _ in range(s):
        line = list(map(int, file.readline().split(" ")))
        morids.update({line[0] : line[2:]})
    g.morids = morids
    
    g.start_position = int(file.readline())

    recipes = []
    for recipe in morids.values():
        recipes.extend(recipe)

    recipes = list(dict.fromkeys(recipes)) # Remove Duplicates From a Python List 
    g.recipes = recipes
    return g

g1 = create_graph("input.txt")
g2 = create_graph("input2.txt")
g3 = create_graph("input3.txt")


## Create initial state

In [5]:
def create_initial_state(g :Graph) -> State:
    s = State()
    s.position = g.start_position
    return s

initial_state1 = create_initial_state(g1)
initial_state2 = create_initial_state(g2)
initial_state3 = create_initial_state(g3)
    

TypeError: State.__init__() missing 1 required positional argument: 'rocky_nodes'

## Transition module and goal state implementation 

In [ ]:
def reached_goal(s :State, g :Graph) -> bool:
    return len(g.morids) == len(s.morids_seen)

def transition(s :State, g :Graph, p :int) -> State:
    next_state = copy.deepcopy(s)
    next_state.prev_state = s
    next_state.cost += 1
    if s.rocky_remain != 0:
        next_state.rocky_remain -= 1
    else:
        next_state.position = p
        if g.is_rocky(p):
            next_state.rocky_remain = next_state.rocky_nodes[p]
            next_state.rocky_nodes[p] += 1

        elif(g.is_recipes() and p not in s.recipes_seen):
            next_state.recipes_seen.append(p)

        elif(g.is_morid() and p not in s.morids_seen and Counter(g.morids[p]) == Counter(s.recipes_seen)):
            next_state.morids_seen.append(p)
            
    return next_state
    

SyntaxError: invalid syntax (1500345839.py, line 16)

In [ ]:
# x = [1,2,3]

# print(12 in x)

False


In [ ]:
#so None works!
# aa = None

# if aa == 12:
#     print("ok")
# else:
#     print("ok2")

ok2


In [9]:
aa = [1,2,3]
bb = [3,2,1]

Counter(aa) == Counter(bb)

print(aa)
print (bb)

# a = 67 

# def func():
#     print(a)

# # x = [12,13,True,"salam"]
# # if x == "salam":
# #     print("yes")
# # print("no")

# # print(range(10))
# func()

# d = {
#     1 : [1,2,3],
#     2 : [1,2,3],
#     9 : [1,2,3],
#     "salam" : 78,
#     4 : [1,2,3],
# }
# d["salam"] += 100
# print(d)

# s = "salam balajan"
# print(s[2:])
# sss = "33"
# b = int(sss.split(" "))
# # print(a)
# print(b)
# type(b)

[1, 2, 3]
[3, 2, 1]


In [ ]:
# d = {
#     1 : [1,2,3],
#     2 : [1,2,3],
#     9 : [1,2,3],
#     4 : [1,2,3],
# }

# l = list(d.keys())
# print(l)
print(type(l))
# x = [1,4,"salam", 7]
# print(dict.fromkeys(x, 0))
# len(d)

# for recipe in d.values():
#     print(recipe)
#     print(type(recipe))



[1, 2, 9, 4]
<class 'list'>


In [ ]:
# How deep copy works in python:
# g = Graph(12)
# h = copy.deepcopy(g)
# print(g.v)
# h.v = 33
# print(g.v)